# 02i – 10. Experiment: LLM-Postprocessing v4: Qwen3-8B mit VTT-nativem Prompt

## Motivation

Alle bisherigen LLM-Ansätze (E38–E43) haben die Transkriptionen auf Zeilen- oder
Block-Ebene verarbeitet. Dieser Ansatz geht einen Schritt weiter:
Das LLM erhält die **vollständige VTT-Datei** als Input und gibt eine
korrigierte VTT-Datei zurück – mit allen Timestamps.

**Neue Idee:** Das LLM soll zuerst intern eine Zusammenfassung des Gesprächs erstellen
und darauf basierend Korrekturen vornehmen (`Chain-of-Thought`-ähnlicher Ansatz).

| Merkmal | `02f`/`02g` | `02i` (dieser Ansatz) |
|---------|-------------|----------------------|
| Granularität | 4–8 Zeilen/Aufruf | Gesamte VTT-Datei/Aufruf |
| Prompt-Strategie | Zeilen-Korrektur | Zusammenfassen → Korrigieren |
| Timestamp-Handling | Nicht im Prompt | Timestamps im Output erhalten |
| Cue-Guard | Token-Guard | `apply_line_level_guard` (strukturbasiert) |
| Input | BL4 E09 VTTs | `output_final_bs12_len20` (späterer Bugfix-Stand) |
| max_new_tokens | 256–2048 | 8192 (ganze Datei) |

## Ergebnis (Vorschau)

WER konnte nicht verbessert werden. Dennoch wird E55 auf allen 25 Sessions getestet
(statt nur dem 5-Session-Dev-Subset) – vgl. `03a2_`.
Es sollte ausprobiert werden, ob diese LLM-Postprocessing Variante 4 vielleicht entgegen der ursprünglichen Annahme zwar nicht auf 5 Sessions eine Verbesserung entfaltet, aber auf 25.

**Hinweis zum Bugfix:** Dieser Lauf wurde **vor dem Bugfix** in `segmentation.py` durchgeführt (`min_duration_off` las fälschlicherweise den Wert von `min_duration_on`). Das ist **gewollt**: Der Bugfix wurde erst nach Abschluss der LLM- und Hyperparameter-Experimente entdeckt. Da der Bugfix allein die WER zunächst verschlechterte, wurde erst in `02j_`/`02k_` die Kombination aus Bugfix + `min_duration`-Optimierung erarbeitet, die schließlich das beste Ergebnis lieferte.

## 1 – GPU-Check & Auswahl

In [1]:
!nvidia-smi

Fri Jan 30 18:07:01 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   28C    P0             90W /  500W |    7093MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Anpassen je nach Verfügbarkeit

## 2 – CUDA-Verifikation

In [3]:
import torch

In [4]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Device 0 name:", torch.cuda.get_device_name(0))
    print("Memory allocated:", torch.cuda.memory_allocated(0) / 1024**3, "GB")

CUDA available: True
CUDA devices: 1
Device 0 name: NVIDIA A100-SXM4-80GB
Memory allocated: 0.0 GB


## 3 – Setup: Imports & Arbeitsverzeichnis

In [5]:
from pathlib import Path
import re
import shutil
from glob import glob
from tqdm.auto import tqdm

project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

from script.pg_utils_experiments import append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

## 4 – Konfiguration
`INPUT_OUTPUT_DIRNAME` verweist auf `output_final_bs12_len20` – das ist der
finale Stand **ohne Bugfix** (erzeugt in `03_`), nicht E09 wie in `02f`–`02h`.
Das hat ansonsten aber keine Auswirkungen, weil es diesselben Konfigurationen wie in E09 sind.

In [7]:
MODEL_NAME = "Qwen/Qwen3-8B"
BASE_DIR = "data-bin/dev"
SESSIONS_GLOB = ["session_40", "session_43", "session_49", "session_50", "session_54"]
INPUT_OUTPUT_DIRNAME = "output_final_bs12_len20"
OUTPUT_OUTPUT_DIRNAME = "output_E55_qwen3_8b_v3"

In [8]:
MAX_NEW_TOKENS = 8192 # Muss groß genug sein für eine ganze VTT-Datei
OVERWRITE = False # False: überspringt bereits vorhandene Output-Dateien (Resume)          
SKIP_IF_MISSING_INPUT = True

## 5 – Modell laden

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto", # HuggingFace wählt besten dtype (bfloat16 auf modernen GPUs)
    device_map="auto",
    trust_remote_code=True,
).eval()

# Fallback: pad_token_id auf eos_token_id setzen falls nicht definiert
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.38it/s]


## 6 – Hilfsfunktionen: Prompt-Bau, VTT-Extraktion, Cue-Level-Guard & Session-Processing

**Kernidee des Prompts:** Das LLM soll zuerst intern eine Zusammenfassung des
Gesprächs erstellen (Chain-of-Thought), diese aber nicht ausgeben.
Auf Basis der Zusammenfassung soll es dann die VTT-Datei mit korrigierten
Wörtern ausgeben – die Timestamps müssen dabei exakt erhalten bleiben.

Da das LLM die gesamte VTT-Datei auf einmal verarbeitet, ist ein
strukturierter Guard wichtig. Dieser prüft jede Cue-Zeile einzeln und
revertiert sie auf das Original falls:
1. Eckige Klammern eingeführt wurden (z.B. `[INAUDIBLE]`)
2. Das Casing geändert wurde (ALL CAPS → Mixed Case)
3. Neue Satzzeichen eingefügt wurden die nicht im Original waren
   
`_resolve_sessions` unterstützt zwei Eingabeformate:
- Liste von Session-Namen (relativ zu `base_dir`)
- Glob-Pattern für automatische Suche

`OVERWRITE=False` ermöglicht ein Resume: bereits verarbeitete VTTs werden übersprungen.

In [10]:
# Regex für Timestamp-Zeilen: '00:01:23.456 --> 00:01:25.789'
_TS_LINE = re.compile(r"\d{2}:\d{2}:\d{2}\.\d{3}\s-->\s\d{2}:\d{2}:\d{2}\.\d{3}")

def _timestamps(vtt_text: str):
    # Extrahiert alle Timestamp-Zeilen aus einer VTT-Datei
    return _TS_LINE.findall(vtt_text)

def _extract_webvtt(raw: str) -> str | None:
    # Extrahiert den VTT-Inhalt aus der LLM-Rohausgabe
    # Entfernt Reasoning-Blöcke, Code-Fences und findet den letzten WEBVTT-Block
    # <think>-Blöcke entfernen (Qwen3 Reasoning-Modus)
    raw = re.sub(r"<think>.*?</think>\s*", "", raw, flags=re.DOTALL | re.IGNORECASE)

    # Code-Fences entfernen (Modell schreibt manchmal ```webvtt ... ```)
    raw = raw.replace("```webvtt", "```").replace("```vtt", "```")
    raw = re.sub(r"```+\s*", "", raw)

    # rfind: nimmt den LETZTEN 'WEBVTT'-Block (falls Modell vorher über VTT schreibt)
    idx = raw.rfind("WEBVTT")
    if idx < 0:
        return None # Kein VTT-Block gefunden

    vtt = raw[idx:].strip() + "\n"
    return vtt


def _build_prompt(vtt_text: str) -> str:
    return (
        "You are working on a transcription of a video. You are only transcribing one speaker "
        "and not a complete dialog. Attached you find a first draft of the transcription. "
        "The draft probably includes mistakes due to rare, unclear, or domain-specific words.\n\n"

        "You should do two things:\n"
        "1. Review the attached transcription and summarise what the transcription is about.\n"
        "2. Based on your own summary, rewrite the transcription and exchange words which might "
        "be wrong due to the context of the overall transcription.\n\n"

         # Chain-of-Thought: Zusammenfassung nur intern nutzen, nicht ausgeben
        "IMPORTANT: Use the summary ONLY internally. Do NOT output the summary. Output ONLY the corrected WEBVTT starting with 'WEBVTT'.\n"
        "Also do NOT output <think> tags or any reasoning.\n\n"

        "Only exchange certain words with better fitting words which may sound similar from a "
        "pronunciation perspective.\n"
        "You MAY also fix missing or wrong small function words (e.g., a/the/to/of/your/I'm/it's) "
        "if it clearly improves grammatical correctness.\n"
        "Do NOT paraphrase or change meaning. Keep the same tone and level of formality.\n"
        "Keep the structure of the attached file exactly as it is. "
        "Keep all timestamps exactly as they are. "
        "The transcription must be complete. Do not leave any parts out. "
        "Do not add new content. Do not hallucinate.\n\n"
        "IMPORTANT: Use the summary ONLY internally. Do NOT output the summary. "
        "Output ONLY the corrected WEBVTT starting with 'WEBVTT'.\n"
        "Also do NOT output <think> tags or any reasoning.\n\n"

        "ADDITIONAL STRICT FORMAT RULES:\n"
        "- Keep the exact casing style of each cue line (if it is ALL CAPS, keep ALL CAPS).\n"
        "- Do NOT add punctuation that wasn't there (no commas, question marks, brackets, etc.).\n"
        "- NEVER insert placeholders like [DEVICE], [INAUDIBLE], [MUSIC], or anything in brackets.\n"
        "- Only replace individual words when highly confident; otherwise keep the original words.\n"
        "- Output ONLY the corrected WEBVTT starting with 'WEBVTT'. No summary, no reasoning.\n\n"
        
        "=== TRANSCRIPTION (WEBVTT) START ===\n\n"
        f"{vtt_text.strip()}\n\n"
        "=== TRANSCRIPTION END ==="
    )


@torch.inference_mode()
def smooth_vtt_with_qwen(vtt_text: str) -> str:
    prompt = _build_prompt(vtt_text)

    messages = [{"role": "user", "content": prompt}]

    # Chat-Template anwenden (kein enable_thinking: System-Prompt statt Flag)
    chat_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([chat_text], return_tensors="pt").to(model.device)

    generated = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS, # 8192: groß genug für eine komplette VTT-Datei
        do_sample=True,
        temperature=0.2, # Sehr niedrig: fast deterministisch, aber leicht variabel
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Nur generierte Tokens dekodieren (Prompt überspringen)
    gen_ids = generated[0][inputs["input_ids"].shape[1]:]
    raw = tokenizer.decode(gen_ids, skip_special_tokens=True)

    cleaned = _extract_webvtt(raw)
    if cleaned is None:
        return vtt_text # Kein VTT-Block → Original zurückgeben
    
    # Cue-Guard: verhindert Klammern, Groß/Kleinschreibungs-Drift, neue Satzzeichen
    cleaned = apply_line_level_guard(vtt_text, cleaned)
    
    # Timestamp-Guard: falls Timestamps verändert wurden → Original zurückgeben
    if _timestamps(cleaned) != _timestamps(vtt_text):
        return vtt_text
    return cleaned


def _resolve_sessions(sessions_spec, base_dir: str | Path | None = None):
    # Löst Sessions-Spec in eine Liste von Pfaden auf
    # Unterstützt: Glob-Pattern (str/Path) oder Liste von Namen/Pfaden
    if isinstance(sessions_spec, (str, Path)):
        return [Path(p) for p in sorted(glob(str(sessions_spec)))]
    if isinstance(sessions_spec, (list, tuple)):
        out = []
        for s in sessions_spec:
            p = Path(s)
            
            # Relative Namen: base_dir davor setzen
            if not p.is_absolute() and base_dir is not None:
                p = Path(base_dir) / p
            out.append(p)
        return out
    raise TypeError(f"sessions_spec must be str/Path or list/tuple, got {type(sessions_spec)}")

def process_sessions(
    sessions_glob,
    input_output_dirname: str,
    output_output_dirname: str,
    overwrite: bool = True,
    base_dir_for_list: str | Path | None = None,
):
    # Verarbeitet alle Sessions: kopiert Input-Verzeichnis und postprocessiert VTTs
    session_dirs = _resolve_sessions(sessions_glob, base_dir=base_dir_for_list)

    print(f"Found {len(session_dirs)} sessions.")
    print("CWD:", Path().resolve())

    missing_session_dirs = [p for p in session_dirs if not p.exists()]
    if missing_session_dirs:
        print("WARNING: these session dirs do not exist (first 5):", missing_session_dirs[:5])

    processed_sessions = 0
    skipped_missing_input = 0
    processed_vtts = 0

    for sdir in tqdm(session_dirs, desc="Sessions"):
        sdir = Path(sdir)
        in_dir = sdir / input_output_dirname
        out_dir = sdir / output_output_dirname

        if not in_dir.exists():
            skipped_missing_input += 1
            continue

        out_dir.mkdir(parents=True, exist_ok=True)
        processed_sessions += 1

        # Nicht-VTT-Dateien direkt kopieren (Metadaten etc.)
        for p in in_dir.iterdir():
            if p.is_file() and p.suffix.lower() != ".vtt":
                dst = out_dir / p.name
                if overwrite or (not dst.exists()):
                    shutil.copy2(p, dst)

         # VTT-Dateien postprocessen
        vtt_files = sorted([p for p in in_dir.iterdir() if p.is_file() and p.suffix.lower() == ".vtt"])
        for vtt_path in tqdm(vtt_files, desc=f"{sdir.name}: VTTs", leave=False):
            dst_path = out_dir / vtt_path.name
            if dst_path.exists() and (not overwrite):
                continue # Resume: bereits verarbeitete Datei überspringen
            vtt_text = vtt_path.read_text(encoding="utf-8", errors="replace")
            fixed = smooth_vtt_with_qwen(vtt_text)
            dst_path.write_text(fixed, encoding="utf-8")
            processed_vtts += 1

    print("Done.")
    print(f"Processed sessions: {processed_sessions}")
    print(f"Skipped (missing input dir '{input_output_dirname}'): {skipped_missing_input}")
    print(f"Processed VTT files: {processed_vtts}")

import re

def _cue_text_lines(vtt_text: str):
    lines = []
    for line in vtt_text.splitlines():
        s = line.strip("\n")
        if not s.strip():
            continue
        if s.strip() == "WEBVTT":
            continue
        if re.match(r"^\d{2}:\d{2}:\d{2}\.\d{3}\s-->\s\d{2}:\d{2}:\d{2}\.\d{3}$", s.strip()):
            continue
        lines.append(s)
    return lines

def _is_all_caps(s: str) -> bool:
    # Prüft ob alle Buchstaben im String Großbuchstaben sind
    letters = [c for c in s if c.isalpha()]
    return bool(letters) and all(c.isupper() for c in letters)

def _valid_line_change(orig: str, new: str) -> bool:
    # Validiert eine einzelne Zeilen-Änderung (orig → new)
    # Gibt False zurück falls die Änderung eine der Guard-Regeln verletzt
    # 1) Keine Klammer-Platzhalter: [INAUDIBLE], [DEVICE] etc.
    if "[" in new or "]" in new:
        return False

    # 2) ALL CAPS-Style erhalten: wenn Original ALL CAPS, darf Output nicht Mixed Case sein
    if _is_all_caps(orig) and not _is_all_caps(new):
        return False

    # 3) Keine neuen Satzzeichen: erlaubt sind Buchstaben, Ziffern, Leerzeichen, Apostrophe
    # und alle Zeichen die bereits im Original vorkamen
    allowed_base = set("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789 '")
    orig_extra = set([c for c in orig if c not in allowed_base]) # Sonderzeichen aus Original
    for c in new:
        if c in allowed_base:
            continue
        if c not in orig_extra:   
            return False  # Neues Satzzeichen → ablehnen

    return True

def apply_line_level_guard(original_vtt: str, corrected_vtt: str) -> str:
    # Wendet den Cue-Guard auf die gesamte VTT-Datei an
    # Ersetzt jede Cue-Zeile die eine Guard-Regel verletzt durch das Original
    orig_text = _cue_text_lines(original_vtt)
    new_text  = _cue_text_lines(corrected_vtt)

    # Zeilenanzahl muss übereinstimmen – sonst gesamte Datei revertieren
    if len(orig_text) != len(new_text):
        return original_vtt if original_vtt.endswith("\n") else (original_vtt + "\n")

    text_i = 0
    out_lines = []
    for line in corrected_vtt.splitlines():
        s = line.strip()
        if not s:
            out_lines.append(line)
            continue
        if s == "WEBVTT" or re.match(r"^\d{2}:\d{2}:\d{2}\.\d{3}\s-->\s\d{2}:\d{2}:\d{2}\.\d{3}$", s):
            out_lines.append(line) # Timestamps unverändert
            continue

        orig_line = orig_text[text_i]
        new_line  = new_text[text_i]

        # Cue-Textzeile: prüfen ob Änderung valide
        if _valid_line_change(orig_line, new_line):
            out_lines.append(new_line) # Korrektur übernehmen
        else:
            out_lines.append(orig_line) # Original behalten
        text_i += 1

    return "\n".join(out_lines).rstrip() + "\n"

## 9 – Postprocessing starten

In [11]:
process_sessions(
    sessions_glob=SESSIONS_GLOB,
    input_output_dirname=INPUT_OUTPUT_DIRNAME,
    output_output_dirname=OUTPUT_OUTPUT_DIRNAME,
    overwrite=OVERWRITE,
    base_dir_for_list=BASE_DIR,
)

Found 5 sessions.
CWD: /home/josch080/Projektgruppe/mcorec_baseline


Sessions: 100%|██████████| 5/5 [44:32<00:00, 534.47s/it]        

Done.
Processed sessions: 5
Skipped (missing input dir 'output_final_bs12_len20'): 0
Processed VTT files: 24


## 10 – Evaluation & Aggregation

In [16]:
EXPERIMENTS = {
    "E55_qwen3_8b_v3": {
        "llm_model": "qwen3_8b", 
        "description": "Qwen3 8B v3"
    }, 
}
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSIONS_GLOB,
    target_csv="results_dev_subset_by_session.csv",
)


########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 11 – Ergebnisanalyse: E55 vs. E09-Baseline

Hier wird `exp`-Spalte (Verzeichnisname) statt `model`-Spalte genutzt,
da `append_eval_results_for_experiments` bei dieser Aufrufform anders befüllt.
Pivot-Tabelle zeigt WER pro Session für E09 und E55 sowie die Delta-Werte.

In [17]:
import pandas as pd
from pathlib import Path

CSV_PATH = Path("results_dev_subset_by_session.csv")

BASELINE_EXP = "output_E09_bs12_len20"
NEW_EXP      = "output_E55_qwen3_8b_v3"

SESSIONS = ["session_40", "session_43", "session_49", "session_50", "session_54"]  # deine 5

df = pd.read_csv(CSV_PATH)

# Falls Timestamp vorhanden: pro (session, exp) den neuesten Eintrag behalten
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = (df.sort_values(["session", "exp", "timestamp"])
            .drop_duplicates(["session", "exp"], keep="last"))

# Auf 5 Sessions und 2 Experimente filtern
df2 = df[df["session"].isin(SESSIONS) & df["exp"].isin([BASELINE_EXP, NEW_EXP])].copy()

# Pivot: Sessions als Zeilen, Experimente als Spalten
piv = df2.pivot_table(index="session", columns="exp", values="avg_speaker_wer", aggfunc="first")

# Sicherstellen, dass beide Spalten existieren
missing_cols = [c for c in [BASELINE_EXP, NEW_EXP] if c not in piv.columns]
if missing_cols:
    raise ValueError(f"Fehlende exp(s) in CSV für diese Sessions: {missing_cols}")

# Δ WER: positiv = E55 besser als E09 (Baseline minus New; niedrigere WER ist besser)
piv["Δ WER (abs)"] = piv[BASELINE_EXP] - piv[NEW_EXP]
piv["Δ WER (rel %)"] = (piv["Δ WER (abs)"] / piv[BASELINE_EXP]) * 100

# Gesamt über 5 Sessions: Mittelwert
baseline_mean = piv[BASELINE_EXP].mean()
new_mean      = piv[NEW_EXP].mean()
delta_abs     = baseline_mean - new_mean
delta_rel     = (delta_abs / baseline_mean) * 100

# Übersichtlich anzeigen
display(
    piv.reset_index()
       .rename(columns={BASELINE_EXP: "WER E09", NEW_EXP: "WER E55"})
       .style.format({
           "WER E09": "{:.4f}",
           "WER E55": "{:.4f}",
           "Δ WER (abs)": "{:+.4f}",
           "Δ WER (rel %)": "{:+.2f}%",
       })
)

print("\n=== Gesamt WER über die 5 Sessions (Mittelwert aus CSV) ===")
print(f"E09  ({BASELINE_EXP}): {baseline_mean:.4f}")
print(f"E55  ({NEW_EXP})     : {new_mean:.4f}")
print(f"Verbesserung (abs)   : {delta_abs:+.4f}")
print(f"Verbesserung (rel)   : {delta_rel:+.2f}%")


exp,session,WER E09,WER E55,Δ WER (abs),Δ WER (rel %)
0,session_40,0.4770,0.4770,+0.0000,+0.00%
1,session_43,0.4688,0.4693,-0.0004,-0.09%
2,session_49,0.4387,0.4402,-0.0015,-0.35%
3,session_50,0.5126,0.5118,+0.0007,+0.14%
4,session_54,0.5799,0.5812,-0.0014,-0.23%



=== Gesamt WER über die 5 Sessions (Mittelwert aus CSV) ===
E09  (output_E09_bs12_len20): 0.4954
E55  (output_E55_qwen3_8b_v3)     : 0.4959
Verbesserung (abs)   : -0.0005
Verbesserung (rel)   : -0.10%


Die WER kontte nicht verbessert werden. Dennoch wird dieses LLM auf allen 25 Sessions getestet.

## 12 – Interpretation & Ausblick

WER konnte durch den VTT-nativen Ansatz nicht verbessert werden.

**Warum scheitert auch dieser Ansatz?**
- **Kontextlimit:** Lange VTT-Dateien nähern sich dem 8192-Token-Limit;
  das LLM sieht möglicherweise nicht die gesamte Datei
- **Halluzinations-Risiko:** Bei ganzen Dateien ist die Wahrscheinlichkeit größer,
  dass das LLM Content erfindet oder Strukturen verändert
- **Cue-Guard als Bremse:** Der Guard verhindert Verschlechterungen,
  aber auch viele mögliche Verbesserungen

**Warum trotzdem auf 25 Sessions testen?**
Die WER-Metrik auf dem 5-Session-Dev-Subset ist nicht immer repräsentativ.
Das vollständige Dev-Set (25 Sessions) gibt ein robusteres Bild.
Ergebnisse dazu in `03a2_results_postprocessing_llm_v3.ipynb`.

**Gesamtfazit LLM-Postprocessing:**
Keiner der getesteten LLM-Ansätze (E38–E55) bringt eine robuste
oder substanzielle WER-Verbesserung. Das beste Ergebnis (E38: −0.0006)
ist statistisch nicht signifikant. Alle weiteren Experimente konzentrieren sich
auf das Bugfix-Parametergitter (`02k_`, `02l_`).
